Dataset: http://archive.ics.uci.edu/ml/datasets/Bag+of+Words

In [1]:
import os
import pandas as pd
from efficient_apriori import apriori
import time
import psutil

In [2]:
def frequent(k,f,name):
   
    out = open("output.txt", "a+")
    start = time.time()
    process = psutil.Process(os.getpid())
    os.chdir('/home/arkaprava/ML/')
    
    #Reading the files
    voc_raw = pd.read_csv('vocab.'+str(name)+'.txt',header= None)
    doc_raw = pd.read_csv('docword.'+str(name)+'.txt',header= None)
    doc_temp = doc_raw.loc[3:]
    
    #Docword of the chosen Text Collection
    doc = pd.DataFrame(doc_temp[0].str.split(' ',2).tolist(),columns=['Doc_id','Word_id','Count'])  
    
    #Vocabulary of the chosen Text Collection
    voc = voc_raw
    voc["Word_id"] = range(1,voc.shape[0]+1)  
    voc.columns = ['words','Word_id']
      
    # Creating dictionary with Word_id as keys and words as values
    dic_voc={}
    for i in range(0,voc.shape[0]):
        dic_voc[i+1] = voc['words'][i]      
    
    # Creating a list of lists(bow) consisting of Word_ids of Doc_id i  in ith list
    dic_doc = doc.groupby('Doc_id')['Word_id'].apply(list).to_dict()
    bow=[]
    for i in dic_doc.keys():
      bow.append(dic_doc[i])
    ans=[]
    
    # Apriori 
    itemsets, _ = apriori(bow, min_support=f)
    try:
     for key in itemsets[k].keys():
            key = list(key)
            for i in range(len(key)):
              key[i] = dic_voc[int(key[i])]     
            ans.append(tuple(key))
    except KeyError:
            print("[]", file = out)
    end = time.time()    
    
    # Outputs       
      
    print("\nName of Text Collection:",str(name).capitalize(), file = out)
    print("\nShape of the Document set of",str(name).capitalize(),":",doc.shape, file = out)
    print("\nShape of the Vocabulary set of",str(name).capitalize(),":",voc.shape, file = out)
    print("\nHere are the all",k,"-itemset of",str(name).capitalize(),'with minimum support',f,':', file=out)     
    print("\n",ans, file=out)
    print('\nComputation time:',(end-start),'Seconds', file=out)     
    print("\nTotal memory used:",process.memory_info().rss,"bytes", file=out)
    print("\n\n ###############################################################\n\n",file = out)
    out.close()
   
frequent(2,0.2,"kos")
